In [105]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

In [106]:
sector = ["semi", "semi_product", "automobile", "bank", "insurance", "finance", "steel", "refinery", "chemistry", "food", "game", "internet", 'battery', 'bio']

# 반도체("삼성전자", "하이닉스")
semi = ["005930.KS", "000660.KS"]
# 반도체 장비("DB하이텍", "한미반도체", "동진쎄미켐", "원익IPS", "신성이엔지", "리노공업", "주성엔지니어링", "솔브레인", "KEC", "하나마이크론")
# semi_product = ["000990.KS", "042700.KS", "005290.KS", "240810.KQ", "011930.KS", "058470.KQ", "036930.KQ", "036830.KQ", "092220.KS", "067310.KQ"]
semi_product = ["000990.KS", "042700.KS"]
# 자동차("현대차", "기아차", "현대모비스")
automobile = ["005380.KS", "000270.KS", "012330.KS"]
# 은행("KB금융", "신한지주", "하나금융지주")
bank = ["105560.KS", "055550.KS", "086790.KS"]
# 보험("삼성생명", "삼성화재")
insurance = ["032830.KS", "000810.KS"]
# 증권("삼성증권", "미래에셋증권", "NH투자증권")
finance = ["016360.KS", "006800.KS", "005940.KS"]
# 철강("포스코홀딩스?", "현대제철")
steel = ["005490.KS", "004020.KS"]
# 정유("sk이노베이션? 지수사", "s-oil", "gs")
refinery = ["096770.KS", "010950.KS", "078930.KS"]
# 화학("롯데케미칼", "lg화학")
chemistry = ["011170.KS", "051910.KS"]
# 음식료("cj제일제당", "농심", "오리온")
food = ["097950.KS", "004370.KS", "271560.KS"]
# 화장품("아모레퍼시픽", "코스맥스", "한국콜마")
cosmetics = ["090430.KS", "192820.KS", "161890.KS"]
# 통신("skt", "kt", "lgu+")
communication = ["017670.KS", "030200.KS", "032640.KS"]
# 전기가스("한국전력", "한국가스공사", "한전kps")
utility = ["015760.KS", "036460.KS", "051600.KS"]
# 인터넷("네이버", "카카오")
internet = ["035420.KS", "035720.KS"]
# 게임("넷마블", "엔씨소프트", "컴투스", "크레프톤", "위메이드")
game = ["251270.KS", "036570.KS", "078340.KQ", "259960.KS", "112040.KQ"]
# 배터리("lg에너지솔루션", "삼성sdi", "에코프로", "에코프로비엠", "포스코퓨처엠")
battery = ["373220.KS", "006400.KS", "086520.KQ", "247540.KQ", "003670.KS"]
# 바이오("한미약품", "유한양행", "대웅제약", "삼성바이오로직스", "셀트리온", "종근당", "sk바이오사이언스")
bio = ["128940.KS", "000100.KS", "069620.KS", "207940.KS", "068270.KS", "001630.KS", "302440.KS"]
# miscellaneous("조선", "운송", "기계", "엔터")
# miscellaneous = ["조선", "운송", "기계", "엔터"]

In [107]:
pxclose = pd.DataFrame()
sector_matrix = pd.DataFrame(columns = sector)

for index, value in enumerate(sector):
   pxclose[value] = yf.download(eval(f'{value}'), interval='1wk', start = '2010-01-01')['Adj Close'].pct_change(fill_method=None).mean(axis='columns').round(2)

pxclose.corr()

[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  7 of 7 completed


,semi,semi_product,automobile,bank,insurance,finance,steel,refinery,chemistry,food,game,internet,battery,bio
semi,1.000000,0.397733,0.336532,0.358981,0.257086,0.446569,0.399384,0.313546,0.361962,0.036370,0.183183,0.224084,0.296635,0.205511
semi_product,0.397733,1.000000,0.263872,0.250005,0.212230,0.361719,0.303947,0.236940,0.284864,0.144577,0.227401,0.194887,0.318087,0.258564
automobile,0.336532,0.263872,1.000000,0.420084,0.425846,0.428169,0.438163,0.401154,0.376538,0.086672,0.161163,0.189024,0.267262,0.197160
bank,0.358981,0.250005,0.420084,1.000000,0.604683,0.679453,0.589485,0.468781,0.442668,0.155512,0.087115,0.146338,0.224920,0.185748
insurance,0.257086,0.212230,0.425846,0.604683,1.000000,0.501975,0.481171,0.379051,0.341054,0.210866,0.098844,0.104759,0.175490,0.166319
finance,0.446569,0.361719,0.428169,0.679453,0.501975,1.000000,0.607239,0.541118,0.561827,0.194821,0.226691,0.256405,0.361412,0.374946
steel,0.399384,0.303947,0.438163,0.589485,0.481171,0.607239,1.000000,0.574035,0.567349,0.147979,0.125282,0.139328,0.305564,0.212288
refinery,0.313546,0.236940,0.401154,0.468781,0.379051,0.541118,0.574035,1.000000,0.622968,0.185960,0.111638,0.157445,0.320927,0.262323
chemistry,0.361962,0.284864,0.376538,0.442668,0.341054,0.561827,0.567349,0.622968,1.000000,0.158407,0.195613,0.208045,0.430835,0.312980
food,0.036370,0.144577,0.086672,0.155512,0.210866,0.194821,0.147979,0.185960,0.158407,1.000000,0.182664,0.239010,0.184710,0.286230
